# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one (or multiple)
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using Unitful
using UnitfulAtomic
using PseudoPotentialData

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    # Use AtomsBuilder to setup aluminium cubic unit cell (4 Al atoms)
    # with provided lattice constant, see AtomsBase integration for details.
    unit_cell = bulk(:Al; a=7.65339u"bohr", cubic=true)
    supercell = unit_cell * (repeat, 1, 1)  # Make a supercell

    # Select standard pseudodojo pseudopotentials, construct an LDA model, discretize
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    pseudopotentials = PseudoFamily("dojo.nc.sr.lda.v0_4_1.oncvpsp3.standard.upf")
    model = model_DFT(supercell; pseudopotentials, functionals=LDA(),
                      temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As expected we obtain the unit cell for `repeat=1`:

In [2]:
aluminium_setup(1)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (24, 24, 24), 13824 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [7.65339   , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 448.29 Bohr³
    
        atoms                : Al₄
        atom potentials      : ElementPsp(Al, "/home/runner/.julia/artifacts/1ea71a84cf375286564538a9cab789991f4bf1f4/Al.upf")
                               ElementPsp(Al, "/home/runner/.julia/artifacts/1ea71a84cf375286564538a9cab789991f4bf1f4/Al.u

and 5-fold as large supercell with `repeat=5`:

In [3]:
aluminium_setup(5)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (96, 24, 24), 55296 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [38.267    , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 2241.5 Bohr³
    
        atoms                : Al₂₀
        atom potentials      : ElementPsp(Al, "/home/runner/.julia/artifacts/1ea71a84cf375286564538a9cab789991f4bf1f4/Al.upf")
                               ElementPsp(Al, "/home/runner/.julia/artifacts/1ea71a84cf375286564538a9cab789991f4bf1f4/Al.

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [4]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355355871462                   -1.09    6.9    189ms
  2   -9.356724631959       -2.86       -1.42    1.5   88.6ms
  3   -9.356980662486       -3.59       -2.68    4.1    209ms
  4   -9.357012268730       -4.50       -3.20    3.4    435ms
  5   -9.357012474675       -6.69       -3.32    4.6    141ms
  6   -9.357012511294       -7.44       -3.44    1.0   80.3ms
  7   -9.357012531622       -7.69       -3.58    1.0   80.1ms
  8   -9.357012541896       -7.99       -3.74    1.0   85.6ms
  9   -9.357012547239       -8.27       -4.06    1.1    100ms


In [5]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -18.73919934319                   -0.97    5.9    602ms
  2   -18.75465431863       -1.81       -1.32    1.0    241ms
  3   -18.79151660349       -1.43       -1.93    4.0    345ms
  4   -18.79231934548       -3.10       -2.21    3.1    338ms
  5   -18.79240158000       -4.08       -2.64    1.4    275ms
  6   -18.79240739586       -5.24       -3.15    2.4    315ms
  7   -18.79240848052       -5.96       -3.33    5.2    506ms
  8   -18.79240850618       -7.59       -3.78    1.2    259ms
  9   -18.79240857659       -7.15       -4.25    2.8    361ms


In [6]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.54994894233                   -0.84    8.1    1.95s
  2   -37.56024087155       -1.99       -1.19    1.9    806ms
  3   -37.56175136914       -2.82       -2.58    5.2    1.19s
  4   -37.56453648119       -2.56       -2.41   12.8    2.09s
  5   -37.56454070319       -5.37       -2.45    4.5    945ms
  6   -37.56455347375       -4.89       -3.30    1.0    737ms
  7   -37.56455382745       -6.45       -3.59    5.9    1.16s
  8   -37.56455403646       -6.68       -4.13    2.8    1.22s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [7]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355246145200                   -1.09    6.9    192ms
  2   -9.356774596401       -2.82       -1.91    1.0   61.7ms
  3   -9.357001742496       -3.64       -2.95    4.5    117ms
  4   -9.357006755085       -5.30       -2.88    3.9    136ms
  5   -9.357011706352       -5.31       -3.29    1.1   71.2ms
  6   -9.357012519001       -6.09       -4.11    1.4   82.9ms


In [8]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.55040856414                   -0.84    8.5    2.31s
  2   -37.55215551936       -2.76       -1.53    1.8    671ms
  3   -31.82259081071   +    0.76       -0.64    8.8    1.54s
  4   -28.95530824207   +    0.46       -0.59    7.8    2.00s
  5   -37.55793672935        0.93       -1.83    7.2    1.39s
  6   -37.05710765640   +   -0.30       -1.20    5.2    1.23s
  7   -37.52754846087       -0.33       -1.72    6.1    1.25s
  8   -37.56086507443       -1.48       -2.01    2.9    1.22s
  9   -37.56393004038       -2.51       -2.31    1.8    751ms
 10   -37.56433180645       -3.40       -2.53    2.5    975ms
 11   -37.56450842175       -3.75       -2.78    2.4    926ms
 12   -37.56452649118       -4.74       -3.01    2.2    919ms
 13   -37.56454576376       -4.72       -3.25    3.8    1.03s
 14   -37.56455077042       -5.30       -3.50    1.9    1.18s
 15   -37

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.